In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
import os
import json

In [13]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [14]:
import torch
print(torch.cuda.is_available())

True


In [15]:
model_pt = "google/gemma-3-1b-pt"
model_ft = "models/gemma-3-1b-ft"
log_dir = "models/logs"
tokenizer = AutoTokenizer.from_pretrained(model_pt)
model = AutoModelForCausalLM.from_pretrained(model_pt)

# Force CPU usage
device = torch.device("cuda")
model.to(device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 25.43 GiB is allocated by PyTorch, and 537.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import json

# Load your dataset
with open("trainingData.json", "r", encoding="utf-8") as f:
    data = json.load(f)

conversations = data["conversations"]
converted = []

# Pair up human-gpt messages
for i in range(0, len(conversations) - 1, 2):
    if conversations[i]["from"] == "human" and conversations[i+1]["from"] == "gpt":
        converted.append({
            "prompt": conversations[i]["value"].strip(),
            "answer": conversations[i+1]["value"].strip()
        })

# Save as a pretty JSON array
with open("megumin_formatted.json", "w", encoding="utf-8") as f:
    json.dump(converted, f, indent=4, ensure_ascii=False)

print(f"✅ Converted {len(converted)} prompt-answer pairs.")


✅ Converted 131 prompt-answer pairs.


In [ ]:
dummy_data = [
    {
        "prompt": "What is the capital of France?",
        "answer": "The capital of France is Paris.",
    },
    {
        "prompt": "Who wrote Hamlet?",
        "answer": "Hamlet was written by William Shakespeare.",
    },
]

with open("megumin_formatted.json", "r") as file:
    training_data = json.load(file)

# Function to format data for training
def format_conversation(example):
    return {"text": f"Prompt: {example['prompt']}\nAnswer: {example['answer']}"}


formatted_data = [format_conversation(d) for d in training_data] #dummy_data
dataset = Dataset.from_list(formatted_data)


def tokenize_function(examples):
    output = tokenizer(
        examples["text"], padding="max_length", truncation=True, max_length=128
    )
    output["labels"] = output["input_ids"].copy()  # <-- Add this!
    return output



tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")

Map: 100%|██████████| 131/131 [00:00<00:00, 8969.36 examples/s]


In [ ]:
# Ensure the required package is installed
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir=model_ft,
        per_device_train_batch_size=1,
        num_train_epochs=1,
        save_strategy="epoch",
        logging_dir=log_dir,
        logging_steps=1,
        save_total_limit=2,
        evaluation_strategy="no",
        report_to="none",  # Disable wandb logging
        fp16=True,
        PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True,
    ),
    train_dataset=tokenized_dataset,
)

trainer.train()

d:\Projects\ByteMentor\venv\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
1,4.848900
2,4.825300
3,4.614700
4,5.014700
5,5.479500


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 25.43 GiB is allocated by PyTorch, and 538.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model.save_pretrained(model_ft)
tokenizer.save_pretrained(model_ft)

('models/gemma-3-1b-ft\\tokenizer_config.json',
 'models/gemma-3-1b-ft\\special_tokens_map.json',
 'models/gemma-3-1b-ft\\tokenizer.json')